In [1]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import time
import numpy as np
import pandas as pd
import re
from time import sleep
from random import randint

(1) Yelp uses GET requests for its search. Using Python or Java (Selenium is OK to use but not required and may be more cumbersome here; I didn't use Selenium), write a program that searches on yelp.com for the top-40 “Donut Shop” in the San Francisco area (no need to verify that the shop is actually selling donuts, just search for “Donut Shop”, top 40 shops according to Yelp's "Recommended" sorting). Save each search result page to disk, “sf_donut_shop_search_page_[PN].htm” (replace [PN] with the page number). Please make sure to pause after loading each page. (25 points)

In [2]:
def q1():
    # Initiating section, getting url, and page keys
    s = HTMLSession()
    url1 = 'https://www.yelp.com/search?find_desc=Donut+Shop&find_loc=San+Francisco&start='
    page = [0,10,20,30] 

    # Get request for each page
    for i,n in zip(page, range(1,5,1)):
        url = str(url1)+str(i)
        r = s.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')

        sleep(randint(3,7))

        # Save files
        with open("sf_donut_shop_search_page_"+str(n)+".html", "w", encoding = 'utf-8') as file:
            # prettify the soup object and convert it into a string  
            file.write(str(soup))

In [250]:
# Saving the files
q1()

(2) Using Python or Java, write new code that opens the search result pages saved in (1) and parses out all shop information (search rank, name, linked URL [this store’s Yelp URL], star rating, number of reviews, store tags, “$” signs, delivery / dine-in tags, and whether you can order through Yelp). Please be sure to skip all “Sponsored Results”. (15 points)

In [3]:
## Function to get the required information
def get_names(page):
    with open(page) as fp:
        soup = BeautifulSoup(fp, "html.parser")

    # Finding the main structure
    structure = soup.find('ul', {'class':'undefined list__09f24__ynIEd'})
        
    # Getting names
    n = [el.get_text() for el in structure.find_all('h3')]
    names = n[2:12]

    # Cleaning names
    names_clean = [l.replace(u'\xa0', u'') for l in names]

    # Putting names in the df
    df['names'] = names_clean
        
    name.append(names_clean)
    
    # Spliting name into name and ranking column
    df[['Rank', 'Name']] = df['names'].str.split('.', 1, expand=True)
    df2 = df.drop('names',1)
    
    
    ## Getting ratings 
    r = [el.get('aria-label') for el in structure.find_all('div')]

    # Using List comprehension to remove none values
    filtered_list = [ ele for ele in r if ele is not None]

    # Filtering just rating rows
    substring = 'star rating'
    # Create boolean to show where there is rating columns
    f_list2 = [i.find(substring) != -1 for i in filtered_list]

    # Apply boolean to the dataframe
    bdf = pd.DataFrame()
    bdf['ratings_bool'] = f_list2
    bdf['ratings_col'] = filtered_list

    # Filtering just rating rows
    ratings = bdf[bdf['ratings_bool'] == True]

    # Getting the correct rates
    rates = ratings.iloc[2:12,1].values
    ratess.append(rates)
    
    
    # Getting Num of Reviews
    nr = [el.get_text() for el in structure.find_all('span', {"class":"reviewCount__09f24__tnBk4 css-1e4fdj9"})]
    reviews.append(nr[2:12])
    

    ## Getting price
    price = [el.get_text() for el in structure.find_all('p', {"class":"css-1gfe39a"})]
    pc = price[2:12]

    ## Counting the '$'
    price_count = [i.count('$') for i in pc]
    prices.append(price_count)
    
    
    # Tags
    tagss = [el.get_text() for el in structure.find_all('span', {"class":"css-epvm6 display--inline__09f24__c6N_k border-color--default__09f24__NPAKY"})]
    tags.append(tagss[2:12])
    
    
    ## Getting dinning options
    m = [el.get_text() for el in structure.find_all('div', {"class":"container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY"})]
    options.append(m[2:12])


    # Order through Yelp on df
    do['options'] = [re.sub(r'^.*?\xa0more', '', i) for i in m][2:12]
    yelp_order.append([i.count('Start Order') for i in do['options']])
    
    
    ## Links
    qq = structure.find_all('a',{'class':'css-1422juy'}, href = True)
    link.append([qq[i]['href'] for i in range(5,15)])

In [4]:
## Cleaning data and putting in the final Df
df = pd.DataFrame()
do = pd.DataFrame()
name = []
ratess = []
reviews = []
prices = []
tags = []
options = []
yelp_order = []
link = []

pages = ["sf_donut_shop_search_page_1.html","sf_donut_shop_search_page_2.html","sf_donut_shop_search_page_3.html","sf_donut_shop_search_page_4.html"]

for i in pages:
    get_names(i)

# Clean names
names_df = pd.DataFrame()
names_df['names'] = name
names_df2 = names_df.explode('names').reset_index()

# Spliting name into name and ranking column
names_df2[['Rank', 'Name']] = names_df2['names'].str.split('.', 1, expand=True)
names_df2 = names_df2.drop(['names','index'],1)

# Clean Rates
rates_df = pd.DataFrame()
rates_df['rating'] = ratess
rates_df2 = rates_df.explode('rating').reset_index()

# Joining in the names_df - Ranking Name Ratings df
names_df2['ratings'] = rates_df2['rating']

# Clean reviews
reviews_df = pd.DataFrame()
reviews_df['reviews'] = reviews
reviews_df2 = reviews_df.explode('reviews').reset_index()

# Joining in the names_df - Ranking; Name; Ratings; Reviews
names_df2['reviews'] = reviews_df2['reviews']
names_df2

# Clean prices
prices_df = pd.DataFrame()
prices_df['prices'] = prices
prices_df2 = prices_df.explode('prices').reset_index()

# Joining in the names_df - Ranking; Name; Ratings; Reviews; Price
names_df2['prices'] = prices_df2['prices']

# Clean tags
tags_df = pd.DataFrame()
tags_df['tags'] = tags
tags_df2 = tags_df.explode('tags').reset_index()

# Joining in the names_df - Ranking; Name; Ratings; Reviews; Price; Tags
names_df2['tags'] = tags_df2['tags']

## Cleaning the options
options_df = pd.DataFrame()
options_df['options'] = options
options_df2 = options_df.explode('options').reset_index()
options_df2['options'] = [re.sub(r'^.*?\xa0more', '', i) for i in options_df2['options']]
p = [i.replace('Start Order',"") for i in options_df2['options']]

# Joining in the names_df - Ranking; Name; Ratings; Reviews; Price; Tags; Dining Options
names_df2['options'] = p
names_df2

# Clean Yelp Order
order_df = pd.DataFrame()
order_df['yelp_order'] = yelp_order
order_df2 = order_df.explode('yelp_order').reset_index()

# Joining in the names_df - Ranking; Name; Ratings; Reviews; Price; Order
names_df2['yelp_order'] = order_df2['yelp_order']
names_df2

# Clean Link
link_df = pd.DataFrame()
link_df['link'] = link
link_df2 = link_df.explode('link').reset_index()

# Joining in the names_df - Ranking; Name; Ratings; Reviews; Price; Order; Link
names_df2['link'] = link_df2['link']
print(names_df2)

/var/folders/vd/d1gk2l750jl53z2v384n4vvc0000gn/T/ipykernel_12151/1268898509.py:23: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df2 = df.drop('names',1)


   Rank                           Name          ratings reviews prices  \
0     1     Bob’s Donuts & Pastry Shop  4.5 star rating    3328      1   
1     2  Uncle Benny’s Donut and Bagel  4.5 star rating     389      2   
2     3      Twisted Donuts and Coffee  4.5 star rating     508      1   
3     4                    Donut World    4 star rating     275      1   
4     5                All Star Donuts    4 star rating     280      1   
5     6                   Bob’s Donuts  4.5 star rating     117      1   
6     7                   Happy Donuts    4 star rating     269      1   
7     8                   Glaze Donuts    4 star rating      89      1   
8     9          Dynamo Donut & Coffee    4 star rating    1945      1   
9    10             Rolling Pin Donuts    4 star rating     782      1   
10   11            Mochill Mochi Donut  4.5 star rating     820      0   
11   12                The Jelly Donut  4.5 star rating     271      1   
12   13                  Golden Donuts

/var/folders/vd/d1gk2l750jl53z2v384n4vvc0000gn/T/ipykernel_12151/3941598375.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  names_df2 = names_df2.drop(['names','index'],1)
